In [ ]:
#Import Libraries
import cx_Oracle
import pandas as pd

In [ ]:
#Connect to Database
connstr = "user/password@abcxyz.com:1521/orcl"
connection = cx_Oracle.connect(connstr)



In [ ]:
# 7 Based on attached table structure, write a query to return the last 5 order ids (by order date) from the orders table

sql1 = """SELECT *
FROM
  (SELECT * FROM ORDERS ORDER BY ORDER_DATE DESC, ORDER_ID DESC
  )
WHERE ROWNUM <= 5
ORDER BY ROWNUM"""

df1 = pd.read_sql(sql1, con=connection)
df1

In [ ]:
#8  Based on attached table structure, write a query to return, by month, 
#   the unique number of events that had orders between 1/1/2017 and 12/31/2017 inclusive 

sql2 = """SELECT TO_CHAR(ORDER_DATE,'YYYY/MM') YEAR_MONTH ,
  COUNT(DISTINCT EVENT_ID) AS UNIQUE_EVENT_CT
FROM ORDERS
WHERE ORDER_DATE BETWEEN '01-JAN-2017' AND '31-DEC-2017'
GROUP BY TO_CHAR(ORDER_DATE,'YYYY/MM')
ORDER BY 1"""

df2 = pd.read_sql(sql2, con=connection)
df2

In [ ]:
#9 Based on the attached table structure, write a query to return the total revenue, 
#  count of unique orders by event_name that occured between 3/15/2018 and 4/15/2018 inclusive

sql3 = """SELECT EV.EVENT_ID,
  MAX(EV.EVENT_NAME)         AS EVENT_NAME,
  COUNT(DISTINCT O.ORDER_ID) AS CT_DISTINCT_ORDER_ID,
  SUM(O.ORDER_TOTAL)
FROM ORDERS O,
  EVENT EV
WHERE O.EVENT_ID = EV.EVENT_ID
AND EV.EVENT_DATE BETWEEN '15-MAR-2018' AND '15-APR-2018'
GROUP BY EV.EVENT_ID"""

df3 = pd.read_sql(sql3, con=connection)
df3

In [ ]:
#10 Based on the attached table structure, write a query to return a list of customer_ids 
#   and the number of orders by customers who have placed more than 5 orders between 3/15/2018 and 4/15/2018 inclusive

sql4 = """SELECT O.CUSTOMER_ID,
  COUNT(0)CT_ORDERS
FROM ORDERS O
WHERE O.ORDER_DATE BETWEEN '15-MAR-2018' AND '15-APR-2018'
GROUP BY O.CUSTOMER_ID
HAVING COUNT(1) > 5"""

df4 = pd.read_sql(sql4, con=connection)
df4



In [ ]:
#11 Based on the attached table structure, return the email_address of the customer along with the event_name 
#   and order_date of the most recent event all customers purchased

sql5 = """WITH TEMP_ORD AS
  (SELECT O.CUSTOMER_ID,
    O.EMAIL_ADDRESS,
    O.EVENT_ID,
    EV.EVENT_NAME,
    O.ORDER_DATE,
    ROW_NUMBER() OVER (PARTITION BY CUSTOMER_ID ORDER BY ORDER_DATE DESC) AS ROW_NUM
  FROM ORDERS O,
    EVENT EV
  WHERE EV.EVENT_ID = O.EVENT_ID
  )
SELECT EMAIL_ADDRESS,
  EVENT_NAME,
  ORDER_DATE
FROM TEMP_ORD
WHERE ROW_NUM = 1"""

df5 = pd.read_sql(sql5, con=connection)
df5


In [ ]:
#12 Based on the attached table structure, write a query to return the event_name, number of orders placed during the
#   onsale_date, number of orders placed non-onsale dates for each event in January 2018

sql6 = """SELECT EV.EVENT_ID,
  MAX(EV.EVENT_NAME) AS EVENT_NAME,
  SUM(
  CASE
    WHEN TRUNC(O.ORDER_DATE) = TRUNC(EV.ONSALE_DATE)
    THEN 1
    ELSE 0
  END) ORD_ONSALE_DT_CT,
  SUM(
  CASE
    WHEN TRUNC(O.ORDER_DATE) <> TRUNC(EV.ONSALE_DATE)
    THEN 1
    ELSE 0
  END ) ORD_NON_ONSALE_DT_CT
FROM ORDERS O,
  EVENT EV
WHERE O.EVENT_ID = EV.EVENT_ID
AND EV.EVENT_DATE BETWEEN '01-JAN-2018' AND '31-JAN-2018'
GROUP BY EV.EVENT_ID"""

df6 = pd.read_sql(sql6, con=connection)
df6

In [ ]:
# Close the connection

connection.close()